### AutoGen Introduction

AutoGen AgentChat!

This is a simple file to introduce me to [AutoGen(AG)](https://microsoft.github.io/autogen/stable//index.html), specifically AG AgentChat, a framework for building AI Agents and applications developed by Microsoft.

In [ ]:
# Importing the necessary libraries
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
# Loading the environment variables
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

#### Using Google's Gemini Models.
Noted that Gemini models are currently in experimental state from the decs, this means that not all models are available for used in Autogen AgentsChat, especially the recently released models.

In [ ]:
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.0-flash",
)

#### Applying the message to model

In [ ]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to Mombasa", source="user")
message

#### Introducing the Agent using AutoGen AgentsChat

In [ ]:
from autogen_agentchat.agents import AssistantAgent

travel_agent = AssistantAgent(
    name="travel_assistant_agent",
    model_client=gemini_client,
    system_message="You are a helpful assistant for an airline. You give short, clear answers.",
    model_client_stream=True
)

#### Putting everything together with on_messages

In [ ]:
from autogen_core import CancellationToken

response = await travel_agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

#### Let's make a local database of ticket prices. This will act as a long term memory where the agent can refer to.

In [ ]:
import os
import sqlite3

# Delete DB if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the DB and table
connection = sqlite3.connect("tickets.db")
conn = connection.cursor()
conn.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
connection.commit()
connection.close()

In [ ]:
# Populating db
def save_city_price(city_name, round_trip_price):
    connection = sqlite3.connect("tickets.db")
    conn = connection.cursor()
    conn.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    connection.commit()
    print(f" City {city_name} with round trip price {round_trip_price} added to db.")
    connection.close()

save_city_price("Nairobi", 500)
save_city_price("Nakuru", 350)
save_city_price("Eldoret", 200)
save_city_price("Kisumu", 180)
save_city_price("Mombasa", 290)
save_city_price("Narok", 420)

In [ ]:
# Function to get the price of a city

def get_city_price(city_name: str) -> float | None:
    """ Getting the round trip price to travel to a city in Kenya """
    connection = sqlite3.connect("tickets.db")
    conn = connection.cursor()
    conn.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = conn.fetchone()
    connection.close()
    return result[0] if result else None

In [ ]:
get_city_price("Narok")

#### Now initilizing an agent that has access to tools, specifically the get city price

In [ ]:
from autogen_agentchat.agents import AssistantAgent

travel_assistant_agent = AssistantAgent(
    name="travel_assistant_agent",
    model_client=gemini_client,
    system_message="You are a helpful assistant for an airline. You give short, clear answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

#### Invoking the agent

In [ ]:
response = await travel_assistant_agent.on_messages([message], cancellation_token=CancellationToken())

for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content